In [1]:
import requests
from pprint import pprint 
from html_table_parser import HTMLTableParser 
import pandas as pd 
import numpy as np
import csv
from bs4 import BeautifulSoup as bs
import time

In [38]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
link = "https://www.usagold.com/reference/prices/goldhistory.php"

In [39]:
source = requests.get(link, headers=headers).text
soup = bs(source,"html.parser")

l = []
for li in soup.find("ul", attrs={'id':'goldpriceyearlinks'}):
    li = li.find('a')
    l.append(li)
l2 = []
for i in range(0,len(l)-1,2):
    l2.append(l[i-1])
l = l2[1::]
links, years = [], []
for i in l:
    years.append(i.getText())
    links.append(i["href"])

In [40]:
for i in range(len(links)):
    print(f"{i}) {years[i]} :  {links[i]}")

0) 2020 :  https://www.usagold.com/reference/prices/goldhistory.php?ddYears=2020
1) 2019 :  https://www.usagold.com/reference/prices/goldhistory.php?ddYears=2019
2) 2018 :  https://www.usagold.com/reference/prices/goldhistory.php?ddYears=2018
3) 2017 :  https://www.usagold.com/reference/prices/goldhistory.php?ddYears=2017
4) 2016 :  https://www.usagold.com/reference/goldprices/2016.html
5) 2015 :  https://www.usagold.com/reference/goldprices/2015.html
6) 2014 :  https://www.usagold.com/reference/goldprices/2014.html
7) 2013 :  https://www.usagold.com/reference/goldprices/2013.html
8) 2012 :  https://www.usagold.com/reference/goldprices/2012.html
9) 2011 :  https://www.usagold.com/reference/goldprices/2011.html
10) 2010 :  https://www.usagold.com/reference/goldprices/2010.html
11) 2009 :  https://www.usagold.com/reference/goldprices/2009.html
12) 2008 :  https://www.usagold.com/reference/goldprices/2008.html
13) 2007 :  https://www.usagold.com/reference/goldprices/2007.html
14) 2006 :  

In [41]:
data = []
def scrape_table(url):
    
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    xhtml = requests.get(url, headers=headers).text

    p = HTMLTableParser()
    p.feed(xhtml) 

    table = p.tables[0][2::]
    return p

In [42]:
table = []

for i in range(len(links)):
    
#     length = len(table.tables[0][2::])
    table = scrape_table(links[i]) 
    print(f"{i+1}) Year {years[i]} scrapped , length : {len(table.tables[0][2::])}")
    
    for j in table.tables[0][2::]:
        data.append(j)
        
    time.sleep(10)

1) Year 2020 scrapped , length : 195
2) Year 2019 scrapped , length : 259
3) Year 2018 scrapped , length : 260
4) Year 2017 scrapped , length : 260
5) Year 2016 scrapped , length : 260
6) Year 2015 scrapped , length : 259
7) Year 2014 scrapped , length : 259
8) Year 2013 scrapped , length : 248
9) Year 2012 scrapped , length : 240
10) Year 2011 scrapped , length : 258
11) Year 2010 scrapped , length : 260
12) Year 2009 scrapped , length : 260
13) Year 2008 scrapped , length : 261
14) Year 2007 scrapped , length : 260
15) Year 2006 scrapped , length : 259
16) Year 2005 scrapped , length : 262
17) Year 2004 scrapped , length : 259
18) Year 2003 scrapped , length : 260
19) Year 2002 scrapped , length : 258
20) Year 2001 scrapped , length : 256
21) Year 2000 scrapped , length : 257
22) Year 1999 scrapped , length : 256
23) Year 1998 scrapped , length : 257
24) Year 1997 scrapped , length : 256
25) Year 1996 scrapped , length : 255
26) Year 1995 scrapped , length : 192
27) Year 1994 scrappe

In [43]:
len(data)

13361

In [44]:
data = np.array(data)
length = len(data)
print(length,data.shape)

13361 (13361, 2)


In [45]:
data

array([['30 Sep 2020', '1892.62'],
       ['29 Sep 2020', '1896.80'],
       ['28 Sep 2020', '1882.46'],
       ...,
       ['1968-01-05', '35.14'],
       ['1968-01-04', '35.14'],
       ['1968-01-03', '35.16']], dtype='<U11')

In [46]:
data = np.insert(data, 0, ["Dates","Rates"])
data = data.reshape( (length+1, 2) )
len(data)

13362

In [47]:
with open('gold.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

In [63]:
def get_latest_data():
    
    def latest_year():
    
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
        link = "https://www.usagold.com/reference/prices/goldhistory.php"

        source = requests.get(link, headers=headers).text
        soup = bs(source,"html.parser")

        l = []
        for li in soup.find("ul", attrs={'id':'goldpriceyearlinks'}):
            li = li.find('a')
            l.append(li)
        l2 = []
        for i in range(0,len(l)-1,2):
            l2.append(l[i-1])
        l = l2[1::]
        links, years = [], []
        for i in l:
            years.append(i.getText())
            links.append(i["href"])

        return links
    
    def scrape_table(url):
    
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
        xhtml = requests.get(url, headers=headers).text

        p = HTMLTableParser()
        p.feed(xhtml) 

        table = p.tables[0][2::]
        return p
    
    print("Fetching data......\n")
    
    current_year = latest_year()[0]
    new = scrape_table(current_year)
    
    array = new.tables[0][2:7]
    main_list = []
    for i in range(len(array)):
        main_list.append(array[i][1])
        date = str(array[i][0]).replace(" ","-")
        print(f"{date} : {float(array[i][1])}")
    return main_list

predict = get_latest_data()

Fetching data......

30-Sep-2020 : 1892.62
29-Sep-2020 : 1896.8
28-Sep-2020 : 1882.46
25-Sep-2020 : 1861.33
24-Sep-2020 : 1867.99


In [64]:
predict

['1892.62', '1896.80', '1882.46', '1861.33', '1867.99']